# List of Contents
### 01. Importing libraries and data
### 02. Data wrangling
### 03. Data quality and consistency checks
### 04. Merge customers data with main orders-products data
### 05. Export data

# 01. Importing libraries and data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Import data
path = r'C:\Users\Lawrence\Documents\Career Foundry Projects\Data Immersion\Achievement #4\2023 Instacart Basket Analysis'
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [3]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# 02. Data wrangling

In [4]:
# Drop columns unimportant to analysis
df_customers = df_customers.drop(columns = ['First Name', 'Surnam', 'date_joined'])

In [5]:
df_customers.rename(columns = {'Gender': 'gender','STATE': 'state', 'Age': 'age'}, inplace = True)

In [6]:
df_customers.dtypes

user_id          int64
gender          object
state           object
age              int64
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [7]:
df_customers['user_id'] = df_customers['user_id'].astype('str')
df_customers['user_id'].dtype

dtype('O')

In [8]:
df_customers.describe()

,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


# 03. Data quality and consistency checks

In [9]:
# Check for mixed data types
for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_customers[weird]) > 0:
             print (col)

C:\Users\Lawrence\AppData\Local\Temp\ipykernel_17720\654259506.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\Lawrence\AppData\Local\Temp\ipykernel_17720\654259506.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\Lawrence\AppData\Local\Temp\ipykernel_17720\654259506.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\Lawrence\AppData\Local\Temp\ipykernel_17720\654259506.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(t

In [10]:
# Find missing values in customers.csv
df_customers.isnull().sum()

user_id         0
gender          0
state           0
age             0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [11]:
# Identify duplicate rows
df_dups = df_customers[df_customers.duplicated()]
df_dups

,user_id,gender,state,age,n_dependants,fam_status,income


There appears to be no missing values or duplicate rows in the customers dataset.

In [12]:
df_customers.head(10)

,user_id,gender,state,age,n_dependants,fam_status,income
0,26711,Female,Missouri,48,3,married,165665
1,33890,Female,New Mexico,36,0,single,59285
2,65803,Male,Idaho,35,2,married,99568
3,125935,Female,Iowa,40,0,single,42049
4,130797,Female,Maryland,26,1,married,40374
5,133128,Female,Kentucky,43,2,married,49643
6,152052,Male,Montana,20,0,single,61746
7,168851,Male,South Carolina,30,0,single,63712
8,69965,Male,Texas,47,1,married,162432
9,82820,Male,Virginia,26,2,married,32072


# 04. Merge customers data with main orders-products data

In [13]:
# Import orders_products data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_aggregated.pkl'))

In [14]:
ords_prods_merge.shape

(32404859, 24)

In [15]:
ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: count, dtype: int64

In [16]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order_number,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price_of_user,spending_level,median_days_since_prior_order,ordering_regularity
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender.,20.5,Non-frequent customer.
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender.,20.5,Non-frequent customer.
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender.,20.5,Non-frequent customer.
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender.,20.5,Non-frequent customer.
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender.,20.5,Non-frequent customer.


In [17]:
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_the_week',
       'order_hour_of_day', 'days_since_prior_order_number', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', '_merge', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_price_of_user', 'spending_level', 'median_days_since_prior_order',
       'ordering_regularity'],
      dtype='object')

In [18]:
ords_prods_merge.dtypes

order_id                            int64
user_id                             int64
order_number                        int64
orders_day_of_the_week              int64
order_hour_of_day                   int64
days_since_prior_order_number     float64
product_id                          int64
add_to_cart_order                   int64
reordered                           int64
product_name                       object
aisle_id                            int64
department_id                       int64
prices                            float64
_merge                           category
price_range_loc                    object
busiest_day                        object
busiest_days                       object
busiest_period_of_day              object
max_order                           int64
loyalty_flag                       object
avg_price_of_user                 float64
spending_level                     object
median_days_since_prior_order     float64
ordering_regularity               

In [19]:
# Adjust data types for index columns
ords_prods_merge['order_id'] = ords_prods_merge['order_id'].astype('str')
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')

In [21]:
# Drop extraneous columns
ords_prods_merge = ords_prods_merge.drop(columns = ['_merge'])

In [26]:
# Merge the datasets
ords_prods_merge = ords_prods_merge.merge(df_customers, on = 'user_id', indicator = True)

In [27]:
ords_prods_merge.head(10)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order_number,product_id,add_to_cart_order,reordered,product_name,...,spending_level,median_days_since_prior_order,ordering_regularity,gender,state,age,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,Low spender.,20.5,Non-frequent customer.,Female,Alabama,31,3,married,40423,both


In [28]:
ords_prods_merge['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [29]:
# Drop indicator column
ords_prods_merge = ords_prods_merge.drop(columns = ['_merge'])

# 05. Export data

In [30]:
# Export data
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers.pkl'))